In [1]:
import h2o
from h2o.estimators import H2OGeneralizedLowRankEstimator

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_181"; Java(TM) SE Runtime Environment (build 1.8.0_181-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /Users/thomasott/opt/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmp7zmqzn9q
  JVM stdout: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmp7zmqzn9q/h2o_thomasott_started_from_python.out
  JVM stderr: /var/folders/9c/tqhnrz3x207bf8pzjxjhkm100000gn/T/tmp7zmqzn9q/h2o_thomasott_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_thomasott_knaj1h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
# Import the USArrests dataset into H2O:
arrestsH2O = h2o.import_file("USArrests.csv")
arrestsH2O_nan = h2o.import_file("USArrests_nan.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
arrestsH2O.describe()

Rows:50
Cols:4




,Murder,Assault,UrbanPop,Rape
type,real,int,int,real
mins,0.8,45.0,32.0,7.3
mean,7.788,170.76000000000005,65.53999999999999,21.232000000000003
maxs,17.4,337.0,91.0,46.0
sigma,4.355509764209289,83.33766084001708,14.474763400836784,9.36638453105965
zeros,0,0,0,0
missing,0,0,0,0
0,13.2,236.0,58.0,21.2
1,10.0,263.0,48.0,44.5
2,8.1,294.0,80.0,31.0


In [5]:
#Yes, we have 4 missing values in this dataset from USArrests_nan.csv
arrestsH2O_nan.describe()

Rows:50
Cols:4




,Murder,Assault,UrbanPop,Rape
type,real,int,int,real
mins,0.8,45.0,32.0,7.8
mean,7.632653061224488,172.12499999999994,65.53999999999999,21.516326530612254
maxs,17.4,337.0,91.0,46.0
sigma,4.25839307305301,84.67952728036175,14.474763400836784,9.24286542282694
zeros,0,0,0,0
missing,1,2,0,1
0,13.2,236.0,58.0,21.2
1,10.0,263.0,48.0,44.5
2,8.1,294.0,80.0,31.0


In [6]:
# Split the dataset into a train and valid set:
#train, test = arrestsH2O_nan.split_frame(ratios=[.8], seed=1234)
train = arrestsH2O_nan

In [7]:
# Build and train the model:
glrm_model = H2OGeneralizedLowRankEstimator(k=4,
                                            loss="quadratic",
                                            gamma_x=0.5,
                                            gamma_y=0.5,
                                            max_iterations=700,
                                            recover_svd=True,
                                            init="SVD",
                                            transform="standardize")

In [8]:
glrm_model.train(training_frame=train)

glrm Model Build progress: |██████████████████████████████████████████████| 100%


In [9]:
glrm_model.model_performance()


ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
RMSE: NaN
Sum of Squared Error (Numeric): 3.01194931068139e-13
Misclassification Error (Categorical): 0.0


In [10]:
out = glrm_model.predict(train)
out

glrm prediction progress: |███████████████████████████████████████████████| 100%


reconstr_Murder,reconstr_Assault,reconstr_UrbanPop,reconstr_Rape
1.30738,0.754315,-0.520907,-0.0342239
0.555925,1.07316,-1.21176,2.48664
0.109747,1.43925,0.99898,1.02605
0.274129,0.21109,-1.07359,-0.21815
0.321095,1.22668,1.75892,2.06469
0.0627812,0.376419,0.860809,1.85913
-1.01744,-0.733648,0.791723,-1.12696
-0.40688,0.777933,0.446294,-0.618458
-4.44089e-16,1.92343,0.99898,1.12343
2.29367,0.459084,-0.382735,0.463457


In [11]:
#Missing Values in the Train Set
train.describe()

Rows:50
Cols:4




,Murder,Assault,UrbanPop,Rape
type,real,int,int,real
mins,0.8,45.0,32.0,7.8
mean,7.632653061224488,172.12499999999994,65.53999999999999,21.516326530612254
maxs,17.4,337.0,91.0,46.0
sigma,4.25839307305301,84.67952728036175,14.474763400836784,9.24286542282694
zeros,0,0,0,0
missing,1,2,0,1
0,13.2,236.0,58.0,21.2
1,10.0,263.0,48.0,44.5
2,8.1,294.0,80.0,31.0


In [12]:
#Missing Values Imputed
out.describe()

Rows:50
Cols:4




,reconstr_Murder,reconstr_Assault,reconstr_UrbanPop,reconstr_Rape
type,real,real,real,real
mins,-1.6045144128336395,-1.5012483428149914,-2.317136320035535,-1.483990721831488
mean,4.215378046623641e-16,6.753625436672905e-16,5.681219383824043e-16,-8.951173136040325e-16
maxs,2.293669647497551,1.9470467690983042,1.758923396186785,2.6489267504556384
sigma,0.9897433186107868,0.9793792286287207,1.0,0.9897433186107868
zeros,0,0,0,0
missing,0,0,0,0
0,1.307382114160744,0.7543145557310341,-0.5209066145816317,-0.034223859825009306
1,0.555924945904129,1.0731637612846607,-1.2117641936023638,2.4866394151564055
2,0.10974725225176346,1.4392498861795655,0.9989800592639793,1.0260533974633113
